In [ ]:
##### 데이터 다운로드
!pip install datasets   # Package install

from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

In [ ]:
##### 코드 상단에서 KoNLPy, Mecab 설치 명령어 실행
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
##### Mecab 동작 확인코드 실행
# from konlpy.tag import Mecab  # KoNLPy를 통해 Mecab 패키지 import
# mecab = Mecab()

# sentence = "실용자연어처리 실습 진행중 이에요. 수업을 시작 할게요"
# print(mecab.morphs(sentence)) # 형태소 출력
# print(mecab.nouns(sentence))  # 명사 출력
# print(mecab.pos(sentence))    # 형태소와 형태소 태그 출력

In [ ]:
##### Word embedding 확인
import gensim
import numpy as np

# load word2vec embedding
embedding_model = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/word2vec/word2vec')
# 얘가 좀 대단한 거 같은데, 얘가 룩업테이블 만드는 거 같은데

In [ ]:
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [ ]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#print(train_data)
#print(dev_data)
#print(test_data)

# mecab 형태소 분석기 사용
from konlpy.tag import Mecab  # KoNLPy를 통해 Mecab 패키지 import
mecab = Mecab()

# 형태소 단위로 Tokenization 된 텍스트를 {train/dev/test}_data에 저장
train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})

print(train_data[0])

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}


In [ ]:
##### 훈련세트에서 훈련하고 평가하기
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 룩업테이블을 정의한다.
# load한 embedding 관련 변수
emb_words = embedding_model.wv.index_to_key # 사전 학습 된 워드 임베딩 단어들 #word2vec에 있는 단어들 가져오는 것
emb_vectors = embedding_model.wv.vectors # 사전 학습 된 워드 임베딩 벡터들 # 그 단어들의 벡터들을 가져 오는 것
emb_dim = embedding_model.vector_size # 사전 학습 된 워드 임베딩 사이즈 (N or D) #벡터들의 사이즈를 가져오는 것
emb_stoi = {} # 단어를 key로, 인덱스를 value로 갖는 dictionary # string to index를 줄여서 stoi라고 한다.
max_seq_len = 200 # 최대 길이: 직접 지정하는 고정 값 # 룩업테이블을 볼 때 얼마나 볼지를 정하는 거 같다.

# pad 인덱스: 0 / unk 인덱스 : 1 설정
emb_stoi['<pad>'] = 0
emb_stoi['<unk>'] = 1

# <pad> 단어와 <unk> 단어에 0으로 이루어진 벡터 부여
emb_vectors = np.concatenate((np.zeros(shape=(2, emb_dim)), emb_vectors), axis=0)
# dim == dimension 차원을 말한다.

# 사전 학습 된 워드 임베딩에 등록되어있는 단어에 인덱스 부여
for i, word in enumerate(emb_words):
  emb_stoi[word] = i + 2

# 임베딩 벡터를 텐서 자료형으로 변환
emb_vectors = torch.cuda.FloatTensor(emb_vectors)


def text_to_index(input_data, stoi, max_seq_len):
  all_index = []

  for sample in input_data:
    index_list = []
    for word in sample['text'].split():
      if word in stoi.keys():  # 단어가 lookup table에 검색 가능한 경우
        index_list.append(stoi[word])
      else:  # 단어가 lookup table에 검색 불가능한 경우
        index_list.append(stoi['<unk>'])

    # Padding: 샘플의 길이가 고정 크기의 최대 길이 (L)보다 작은 경우,pad 인덱스를 추가
    if max_seq_len > len(index_list):
      index_list = index_list + [stoi['<pad>']] * (max_seq_len - len(index_list))
    else: # 샘플의 길이가 고정 크기의 최대 길이 (L)보다 큰 경우 고정 크기까지만 처리
      index_list = index_list[:max_seq_len]
    all_index.append(index_list)

  return all_index

# train, dev, test data들을 index로 변환
train_index = text_to_index(train_data, emb_stoi, max_seq_len)
dev_index = text_to_index(dev_data, emb_stoi, max_seq_len)
test_index = text_to_index(test_data, emb_stoi, max_seq_len)
# print(train_index[0]) #테스트 해보았음
# print(train_index[100])


# randomness 제거
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


# MLP 모델의 아키텍쳐(구조) 정의
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, max_seq_len):
    super(MLP, self).__init__()
    # 임베딩 계층
    self.word_embedding = nn.Embedding.from_pretrained(embeddings=emb_vectors) #
    # 각 층에 대한 정의
    self.input_size = input_size
    self.max_seq_len = max_seq_len
    self.layer1 = nn.Linear(max_seq_len * input_size, hidden_size) # nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    # self.layer3 = nn.Linear(int(0.5*hidden_size), output_size)
    self.layer3 = nn.Linear(int(0.5*hidden_size), int(2*hidden_size))
    self.layer4 = nn.Linear(int(2*hidden_size), hidden_size)
    self.layer5 = nn.Linear(hidden_size, output_size)

    # 활성화 함수
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    # 실제(입력) 데이터: x
    # x를 가지고 순전파를 진행
    # print("출력", x)
    word_emb = self.word_embedding(torch.cuda.LongTensor(x))
    word_emb_new = word_emb.view((-1, self.max_seq_len * self.input_size))
    print(word_emb.shape)
    out1 = self.layer1(word_emb_new)
    # out1 = self.layer1(x) #(sum)
    out2 = self.gelu(out1) # hidden_size
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.gelu(out5)
    out7 = self.layer4(out6)
    out8 = self.gelu(out7)
    out9 = self.layer5(out8)
    out10 = self.softmax(out9)
    return out10
    # out5 = self.layer3(out4)
    # out6 = self.softmax(out5)

    # return out6

'''

단어 1개 -> 100차원
[  |   |  |  | 역시 |   |  |  ] 100차원
[  |   |  |  |    |  명작  |  |  ] 100차원
최대 길이 -> 각 세로 길이가 30 (100 * 30차원인가)
배치 -> 128개
텐서는 이렇게 표현 => [128, 30, 100]
[3, 2, 5] => 개수, 세로, 가로 이기때문에


class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MLP, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    self.layer3 = nn.Linear(int(0.5*hidden_size), int(2*hidden_size))
    self.layer4 = nn.Linear(int(2*hidden_size), hidden_size)
    self.layer5 = nn.Linear(hidden_size, output_size)
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.gelu(out1)
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.gelu(out5)
    out7 = self.layer4(out6)
    out8 = self.gelu(out7)
    out9 = self.layer5(out8)
    out10 = self.softmax(out9)

    return out10
'''
# 하이퍼파라미터 셋팅
input_size = emb_dim # len(vectorizer.vocabulary_) # input size
hidden_size = 100 # hidden size
output_size = 2 # output size is 2 (positive/negative)
learning_rate = 0.00002
batch_size = 128
num_epochs = 30

# 모델 초기화
model = MLP(input_size, hidden_size, output_size, max_seq_len)
device = torch.device("cuda") # use GPU
model = model.to(device)

# optimizer, loss function 정의
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

# dev_vectors와 dev_data['label']을 텐서 자료형으로 변환
dev_tensors = dev_index # torch.cuda.FloatTensor(dev_vectors.toarray(), device=device) # 강의자료 11에서 변경
dev_labels = torch.tensor(dev_data['label'], dtype=torch.long, device=device)

# 학습 시작 (총 num_epochs 만큼)
for epoch in range(num_epochs):
  # model을 학습모드로 만든다.
  model.train()

  # 학습을 하면서 보고싶은 숫자 (학습이 잘되는지 확인)
  epoch_loss = 0
  best_accuracy = 0

  # train_vectors를 텐서 자료형으로 변환
  train_tensors = train_index # torch.cuda.FloatTensor(train_vectors.toarray(), device=device)

  # batch size 단위로 학습 진행
  for i in range(0, len(train_tensors), batch_size):

    # batch 단위 데이터 생성
    batch_data = train_tensors[i:i+batch_size] # batch size 크기의 데이터가 batch_data
    batch_labels = torch.tensor(train_data['label'][i:i+batch_size], device=device)

    # 1. 순전파
    outputs = model(batch_data)
    # 2. 오차 계산
    loss = loss_function(outputs, batch_labels)
    # 3. 역전파
    optimizer.zero_grad()
    # 4. 가중치 업데이트
    loss.backward()
    optimizer.step()

    # epoch loss
    epoch_loss += loss.item()

  # 매 epoch마다 dev 성능 측정
  # 모델을 평가하는 모드로 셋팅
  model.eval()
  with torch.no_grad():
    dev_outputs = model(dev_tensors) # dev 데이터의 순전파
    dev_preds = torch.argmax(dev_outputs, axis=1)
    dev_accuracy = torch.sum(dev_preds == dev_labels).item() / len(dev_labels)

    # save best model on dev data
    if dev_accuracy > best_accuracy:
      best_model = model
      best_accuracy = dev_accuracy

  print(f"Epoch {epoch+1}, Accuracy: {dev_accuracy} , loss: {epoch_loss/len(train_tensors)}")

RuntimeError: ignored

In [ ]:
##### 교차 검증을 사용한 평가
# from sklearn.model_selection import cross_val_score

# all_data = train_data['text']+dev_data['text']+test_data['text']      # all_data = train_data + dev_data + test_data
# all_label = train_data['label']+dev_data['label']+test_data['label']  # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것
# all_vectors = vectorizer.transform(all_data)    # all_data를 숫자로 변환
# scores = cross_val_score(svm, all_vectors, all_label, cv=5)  # 5-cross validation

# print("All scores:", scores)  # 전체 점수 출력
# print("Average: {:.2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std()))  # 표준편차 출력

In [ ]:
##### 그리드 탐색
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'max_iter':[500, 1000, 5000], 'C': [1, 10, 100]}]        # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# grid_search = GridSearchCV(svm, param_grid, cv=3)     # 하이퍼파라미터 서치 할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])   # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score'])     # 파라미터 서치 결과
# print(grid_search.best_params_)   #최고의 모델 파라미터

# #파라미터 서치 결과 출력
# for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
#   print(mean_score, params)

In [ ]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score # Accuracy 측정 함수 import
final_model = best_model # MLP 모델
test_tensors = test_index # torch.cuda.FloatTensor(test_vectors.toarray(), device=device)
test_outputs = final_model(test_tensors) # 최종 모델로 test 데이터 예측
pred_results = torch.argmax(test_outputs, axis=1)
accuracy = accuracy_score(test_data['label'], pred_results.tolist()) # 정확도 측정
print("Accuracy: {:.2f}%".format(accuracy*100)) # 정확도 출력